In [5]:
%pip install face_recognition

Defaulting to user installation because normal site-packages is not writeable
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
Failed to build dlib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [94 lines of output]
      <string>:234: SyntaxWarning: invalid escape sequence '\('
      <string>:235: SyntaxWarning: invalid escape sequence '\('
      <string>:236: SyntaxWarning: invalid escape sequence '\('
      running bdist_wheel
      running build
      running build_ext
      <string>:163: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
      Building extension for Python 3.12.2 (tags/v3.12.2:6abddd9, Feb  6 2024, 21:26:36) [MSC v.1937 64 bit (AMD64)]
      Invoking CMake setup: 'cmake C:\Users\Anon Hossain\AppData\Local\Temp\pip-install-i2zr9un5\dlib_f25c24b2b522400fbf034705c973b993\tools\python -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=C:\Users\Anon Hossain\AppData\Local\Temp\pip-install-i2zr9un5\dlib_f25c24b2b522400fbf034705c973b993\build\lib.win-amd64-cpython-312 -DPYTHON_EXECUTABLE=c:\Program Files\Py

## Picture Encoding and save to created Database

In [1]:
import face_recognition
import cv2
import os
import numpy as np
import sqlite3

# Path to the folder containing all pictures
IMAGE_FOLDER = "all_pictures"
DB_PATH = "encoded.db"

# Connect to SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Create the table for storing face encodings and image paths
cursor.execute('''CREATE TABLE IF NOT EXISTS face_data (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    image_path TEXT NOT NULL,
                    encoding BLOB NOT NULL)''')
conn.commit()

# Function to encode faces from images in a folder
def encode_faces(image_folder):
    for filename in os.listdir(image_folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(image_folder, filename)

            # Load the image
            img = cv2.imread(image_path)

            # Convert to RGB
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Detect faces
            face_locations = face_recognition.face_locations(rgb_img)
            print(f"Detected {len(face_locations)} face(s) in {filename}")

            # Encode the faces
            face_encodings = face_recognition.face_encodings(rgb_img, face_locations)

            # Save encodings and image path to the database
            for encoding in face_encodings:
                encoding_blob = np.array(encoding).tobytes()
                cursor.execute("INSERT INTO face_data (image_path, encoding) VALUES (?, ?)", (image_path, encoding_blob))
                conn.commit()
                print(f"Saved encoding for {filename}")

    print("Face encoding complete. Encoded faces are saved in the database.")

# Run the function
encode_faces(IMAGE_FOLDER)

# Close the database connection
conn.close()


Detected 6 face(s) in 462544704_1745504702891889_6194984155276007712_n.jpg
Saved encoding for 462544704_1745504702891889_6194984155276007712_n.jpg
Saved encoding for 462544704_1745504702891889_6194984155276007712_n.jpg
Saved encoding for 462544704_1745504702891889_6194984155276007712_n.jpg
Saved encoding for 462544704_1745504702891889_6194984155276007712_n.jpg
Saved encoding for 462544704_1745504702891889_6194984155276007712_n.jpg
Saved encoding for 462544704_1745504702891889_6194984155276007712_n.jpg
Detected 2 face(s) in 9789c2bb-966d-4255-86bb-16bf460a5bf7.jpeg
Saved encoding for 9789c2bb-966d-4255-86bb-16bf460a5bf7.jpeg
Saved encoding for 9789c2bb-966d-4255-86bb-16bf460a5bf7.jpeg
Detected 4 face(s) in aab40ba8-a545-4237-a683-4dabf21ecb70.jpeg
Saved encoding for aab40ba8-a545-4237-a683-4dabf21ecb70.jpeg
Saved encoding for aab40ba8-a545-4237-a683-4dabf21ecb70.jpeg
Saved encoding for aab40ba8-a545-4237-a683-4dabf21ecb70.jpeg
Saved encoding for aab40ba8-a545-4237-a683-4dabf21ecb70.jpeg

## Match Pictures and show to result folder

In [2]:
import face_recognition
import cv2
import numpy as np
import sqlite3
import os
import shutil

# Paths
TARGET_FOLDER = "target_picture"
DB_PATH = "encoded.db"
RESULT_FOLDER = "result_pictures"

# Create the result_pictures folder if it doesn't exist
os.makedirs(RESULT_FOLDER, exist_ok=True)

# Connect to SQLite database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Function to match faces from target photos against the database
def match_faces(target_folder, tolerance=0.6):
    for filename in os.listdir(target_folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(target_folder, filename)

            # Load the target image
            target_img = cv2.imread(image_path)
            rgb_target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)

            # Detect faces in the target image
            target_face_locations = face_recognition.face_locations(rgb_target_img)
            print(f"Detected {len(target_face_locations)} face(s) in {filename}")

            # Encode the detected faces
            target_face_encodings = face_recognition.face_encodings(rgb_target_img, target_face_locations)

            if not target_face_encodings:
                print(f"No faces found in {filename}.")
                continue

            for target_encoding in target_face_encodings:
                # Retrieve all stored encodings from the database
                cursor.execute("SELECT image_path, encoding FROM face_data")
                rows = cursor.fetchall()

                found_match = False
                for row in rows:
                    stored_image_path = row[0]
                    stored_encoding_blob = row[1]
                    stored_encoding = np.frombuffer(stored_encoding_blob, dtype=np.float64)

                    # Compare the target encoding with stored encodings
                    matches = face_recognition.compare_faces([stored_encoding], target_encoding, tolerance=tolerance)

                    if True in matches:
                        print(f"Match found: {stored_image_path}")
                        found_match = True

                        # Copy the matched image to the result_pictures folder
                        if os.path.exists(stored_image_path):
                            dest_path = os.path.join(RESULT_FOLDER, os.path.basename(stored_image_path))
                            shutil.copy(stored_image_path, dest_path)
                            print(f"Copied {stored_image_path} to {RESULT_FOLDER}")

                if not found_match:
                    print(f"No matches found for the face in {filename}.")

# Run the function
match_faces(TARGET_FOLDER)

# Close the database connection
conn.close()


Detected 1 face(s) in photo_2024-12-28_11-33-41.jpg
Match found: all_pictures\462544704_1745504702891889_6194984155276007712_n.jpg
Copied all_pictures\462544704_1745504702891889_6194984155276007712_n.jpg to result_pictures
Match found: all_pictures\aab40ba8-a545-4237-a683-4dabf21ecb70.jpeg
Copied all_pictures\aab40ba8-a545-4237-a683-4dabf21ecb70.jpeg to result_pictures
Match found: all_pictures\photo_4_2024-08-21_17-46-00.jpg
Copied all_pictures\photo_4_2024-08-21_17-46-00.jpg to result_pictures
Match found: all_pictures\photo_6109147824290641641_y.jpg
Copied all_pictures\photo_6109147824290641641_y.jpg to result_pictures
